In [6]:
from selenium import webdriver
import time
from selenium.webdriver.support.select import Select
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re

In [7]:
#!pip install chromedriver-py

In [8]:
options = Options()

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disabke-gpu")
options.add_argument("--no-sandbox")

## Past IGOs

In [78]:
titles = []
symbols= []
token_metrics = []
presale_info = []
ATH_ROI = []
ATH_value = []
basic_info = []

In [79]:
def open(link):
    try:
        time.sleep(5)
        browser.get(link)
    except:
        print('no internet connection')

In [ ]:
browser = webdriver.Chrome(executable_path="C:\webdrivers\chromedriver.exe", options = options)

def OpenProjectLink():
    
    open('https://igo.gamestarter.com/igo')
    
    time.sleep(2)
    
    browser.maximize_window()
    
    time.sleep(2)
    
    cookie = browser.find_element(by = 'xpath', value = '//div[@style = "cursor:pointer"]')
    
    try:
        cookie.click()
    except:
        pass
    
    time.sleep(2)
    
    i= 0
    
    while i <= 7:
        
        try:
            load = browser.find_element(by = 'xpath', value ='//div[@class = " font-bold "]')
            load.click()
            time.sleep(2)
            i +=1
        
        except:
            break
    
    find_length = len(browser.find_elements(by = 'xpath', value ='//a[@class = "absolute top-0 left-0 h-full w-full z-10"]'))
    
    list_count = 1

    while (list_count < find_length):
        
        global ptitles, psymbols, ptoken_metrics, pATH_ROI, pATH_value, ppresale_info, pbasic_info
        
        time.sleep(2)
        
        try:
            find = browser.find_elements(by = 'xpath', value ='//a[@class = "absolute top-0 left-0 h-full w-full z-10"]')
            
            find[list_count].click()
        
        except:
            pass
        
        time.sleep(3)
        
        try:
            title = browser.find_element(by ='xpath', value = '//div[@class = "h6 simple"]').text
            titles.append(title)
        
        except:
            titles.append('N/A')
        
        try:
            symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
            symbols.append(symbol)
            
        except:
            symbols.append('N/A') 
            
        try:
            presale = browser.find_element(by ='xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')
            presale_info.append(presale.text)
            
        except:
            presale_info.append('N/A')
        
        time.sleep(2)
            
        try:
            basic = browser.find_elements(by ='xpath', value = '//div[@class = "igo-txt dark:font-normal"] //p//ul')
            basic_info.append(basic.text)
            
        except:
            basic_info.append('N/A')
            
        try:
            metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
            for metric in metrics:
                token_metrics.append(metric.text)
            
        except:
            token_metrics.append('N/A')
                    
        time.sleep(2)

        #go back
        browser.back()
        time.sleep(2)
        if list_count == find_length-1:
            
            try:
                ATH = browser.find_elements(by ='xpath', value = '//div[@class = "px-2 text-sm text-white font-bold"]')
                for val in ATH:
                    ATH_ROI.append(val.text)   
            except:
                ATH_ROI.append('N/A')
                
            try:
                ATH = browser.find_elements(by ='xpath', value = '//div[@class = "text-black dark:text-white font-bold text-xl"]')
                for val in ATH:
                    ATH_value.append(val.text)
            except:
                ATH_value.append('N/A')
                
            break
        list_count +=1
        
    return "AWESOME"

OpenProjectLink()

In [81]:
#titles, symbols, token_metrics, ATH_ROI, ATH_value, presale_info, basic_info

In [82]:
#working on symbols

tickers = [symbol.replace('/',' ').replace('BUSD',' ').strip() for symbol in symbols]

In [83]:
#working on token metrics

values = [value.replace('\n',' ') for value in token_metrics]
details = [re.sub(r'Private.+','',item) for item in values]
platform_details = [i for i in details if i]

metrics = [item.split(':') for item in platform_details]

raise_amount = []
price = []
lock_up = []

for i in range(1, len(metrics)+1):
    lock_up.append(metrics[-i][-1].strip())
    price.append(metrics[-i][-2])
    raise_amount.append(metrics[-i][-3])

lock_up = lock_up[::-1]
token_price = [re.sub(r'[a-zA-Z].+','',item).strip() for item in price]
token_price = token_price[::-1]
platform_raise = [re.sub(r'[a-zA-Z].+','',item).strip() for item in raise_amount]
platform_raise = platform_raise[::-1]
platform_raise[1] = '$125,000'

In [139]:
#working on presale info

sale_info = [word for line in presale_info for word in line.split('\n')]

price_type = [sale_info[i] for i in range(0,len(sale_info),4)]
sale_period = [sale_info[i] for i in range(1,len(sale_info),4)]
FCFS_period = [sale_info[i] for i in range(2,len(sale_info),4)]
allocation = [sale_info[i] for i in range(3,len(sale_info),4)]

presale_period = [re.sub(r'Sale\s','',item) for item in sale_period]
FCFS = [re.sub(r'FCFS\s','',item) for item in FCFS_period]
base_allocation = [re.sub(r'.*(.+:)\s','',item) for item in allocation]

In [155]:
#dataframe_list = {'at' : titles, 'b': tickers,'k' :token_price, 'g': platform_raise, 'e': lock_up}
#.T
dataframe_list = [titles, tickers, token_price, lock_up, ATH_value, ATH_ROI, platform_raise, presale_period, FCFS, 
                  base_allocation]
labels= ['Name','Symbol','Price','Vesting Period','ATH Value','ATH ROI','Platform Raise','Presale Period','FCFS Period',
         'FCFS Allocation']
dataframe = pd.DataFrame(np.column_stack(dataframe_list), columns = labels)

dataframe

,Name,Symbol,Price,Vesting Period,ATH Value,ATH ROI,Platform Raise,Presale Period,FCFS Period,FCFS Allocation
0,PROJECT SEED,SHILL,$0.025,100% unlocked on TGE,$1.9099,x76,"$50,000","7 Nov, 10:00 – 8 Nov, 10:00 UTC","8 Nov, 09:00 UTC",$30.1042
1,Dark Frontiers,DARK,$0.05,"20% unlocked on TGE, 4 months cliff, then 5% o...",$1.82,x36,"$125,000","5 Nov, 19:00 – 6 Nov, 19:00 UTC","6 Nov, 18:00 UTC",$80.9716
2,BitHotel,BTH,$0.012,"5% TGE, 3 months cliff, then 5% monthly",$0.261,x21,"$100,000","6 Dec, 14:00 – 7 Dec, 14:00 UTC","7 Dec, 13:00 UTC",$29.6432
3,Demole,DMLG,$0.025,"33%, 2 months cliff, then 33% for two months",$0.5227,x20,"$42,500","18 Oct, 10:00 – 19 Oct, 10:00 UTC","19 Oct, 09:00 UTC",$19.9481
4,DYNASTY,DYT,$0.04,"10% unlocked on TGE, then 10% monthly",$0.7242,x18,"$300,000","11 Nov, 14:00 – 12 Nov, 14:00 UTC","12 Nov, 13:00 UTC",$71.2375
5,Moo Monster,MOO,$0.06,"20% unlocked on TGE, 20% monthly",$0.7469,x12,"$150,000","30 Nov, 14:00 – 1 Dec, 14:00 UTC","1 Dec, 13:00 UTC",$25.4915
6,Crypto Fight Club,FIGHT,$0.02,"10% unlocked on TGE, 7.5% monthly",$0.1426,x7,"$350,000","16 Dec, 14:00 – 17 Dec, 14:00 UTC","17 Dec, 13:00 UTC",$51.6823
7,KingdomX,KT,$0.012,"33% unlocked on TGE, 3 month cliff, linear thr...",$0.0725,x6,"$30,000","5 Feb, 14:00 – 6 Feb, 14:00 UTC","6 Feb, 13:00 UTC",$24.2346
8,NINTIA,NINTI,$0.03,"15% TGE, the rest 10.625% linear release every...",$0.2068,x6,"$300,000","17 Jan, 14:00 – 18 Jan, 18:00 UTC","18 Jan, 17:50 UTC",$236.70
9,12 Legions,CTL,$0.025,"20% unlocked on TGE, 1 months cliff, 20% monthly",$0.161,x6,"$120,000","2 Dec, 14:00 – 3 Dec, 14:00 UTC","3 Dec, 13:00 UTC",$25.0375


### Incoming and Next IGOs

In [17]:
#countdown_front = '//div[@class = "flex flex-row space-x-2"]'
#basic_metrics = '//div[@class = "igo-txt dark:font-normal"] //p//ul'

In [25]:
titles = []
symbols= []
token_metrics = []
presale_info = []
basic_info = []
timer = []
requirements = []
pools = []

In [ ]:
pools

In [28]:
browser = webdriver.Chrome(executable_path="C:\webdrivers\chromedriver.exe", options = options)

def open(link):
    try:
        time.sleep(5)
        browser.get(link)
    except:
        print('no internet connection')
        

def OpenProjectLink():
    time.sleep(5)
    
    open('https://igo.gamestarter.com/igo')
    
    time.sleep(5)
    
    browser.maximize_window()
    
    time.sleep(5)
    
    cookie = browser.find_element(by = 'xpath', value = '//div[@style = "cursor:pointer"]')
    
    try:
        cookie.click()
    except:
        pass
    
    #timer = browser.find_element(by = 'xpath', value = '//div[@class = "flex flex-row space-x-2"]').text
    global titles, symbols, timer, requirements, token_metrics, pools, presale_info, basic_info
    
    presale = browser.find_element(by = 'xpath', value = '//div[@class = "w-full flex lg:flex-row flex-col items-center"]//a')
    time.sleep(2)
    presale.click()
    time.sleep(8)
    
    title = browser.find_element(by ='xpath', value = '//div[@class = "h6 simple"]').text
    titles.append(title)
    
    symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
    symbols.append(symbol)

    req = browser.find_element(by = 'xpath', value = '//p[@class = "text-lg font-normal"]')
    requirements.append(req.text)
    
    metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
    for metric in metrics :
        token_metrics.append(metric.text)
    time.sleep(2)
    
    allocation = browser.find_element(by = 'xpath', value = '//div[@class = "flex justify-between items-center font-normal"]')
    pools.append(allocation.text)
    
    countdown = browser.find_element(by = 'xpath', value = '//p[@class = "text-block-190 font-semibold"]')
    timer.append(countdown.text)
    
    presale = browser.find_element(by = 'xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')
    presale_info.append(presale.text)
    
    browser.back()
    time.sleep(3)

    find_length = len(browser.find_elements(by = 'xpath', value ='//div[@class = "p-6 lg:w-1/3 w-full"]'))
    
    list_count = 0

    while (list_count < find_length):
        
        try:
            find = browser.find_elements(by = 'xpath', value ='//div[@class = "p-6 lg:w-1/3 w-full"]')
            
            find[list_count].click()
        
        except:
            pass
        
        time.sleep(3)
        
        try:
            title = browser.find_element(by ='xpath', value = '//div[@class = "h6 simple"]').text
            titles.append(title)
        
        except:
            titles.append('N/A')
        
        try:
            symbol = browser.find_element(by ='xpath', value = '//div[@class = "font-semibold text-base"]').text
            symbols.append(symbol)
            
        except:
            symbols.append('N/A')
            
        try:
            req = browser.find_element(by = 'xpath', value = '//p[@class = "text-lg font-normal"]')
            requirements.append(req.text)
        
        except:
            requirements.append('N/A')
            
        try:
            metrics = browser.find_elements(by ='xpath', value = '//table[@width ="100%"]')
            for metric in metrics :
                token_metrics.append(metric.text)
            
        except:
            token_metrics.append('N/A')
        
        try: 
            allocation = browser.find_element(by = 'xpath', value = '//div[@class = "flex justify-between items-center font-normal"]')
            pools.append(allocation.text)
            
        except:
            pools.append('N/A')
        
        try:
            presale = browser.find_element(by = 'xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')
            presale_info.append(presale.text)
            
        except:
            presale_info.append('N/A')
                                            
            
        time.sleep(5)

        #go back
        browser.back()
        time.sleep(3)
        
        if list_count == find_length-1:
            break
        list_count +=1
        
    return "cool"
        
OpenProjectLink()

'cool'

### Testing Space

In [ ]:
browser = webdriver.Chrome(executable_path="C:\webdrivers\chromedriver.exe", options = options)
open('https://igo.gamestarter.com/igo')
time.sleep(2)
browser.maximize_window()

cookie = browser.find_element(by = 'xpath', value = '//div[@style = "cursor:pointer"]')
try:
    cookie.click()
except:
    pass

i= 0
while i <= 7:
    try:
        load = browser.find_element(by = 'xpath', value ='//div[@class = " font-bold "]')
        load.click()
        time.sleep(2)
        i +=1
    except:
        break

time.sleep(3)
find = browser.find_element(by = 'xpath', value ='//a[@class = "absolute top-0 left-0 right-0 bottom-0"]')
find.click()
time.sleep(5)
presale_info = []
presale = browser.find_element(by ='xpath', value = '//div[@class = "dark:text-dark-text font-semibold"]')

In [165]:
timer

'10\nd\n11\nh\n49\nm\n16\nSeconds'